In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import os
import glob

from ssd_training import plot_log

In [ ]:
experiments = !ls -1 ./checkpoints
print(experiments.n)

### Log

In [ ]:
checkdir = './checkpoints/' + experiments[-1]

names = [
    'loss',
    #'lr', 
    'seg_conf_loss', 'pos_seg_conf_loss', 'neg_seg_conf_loss',
    'seg_loc_loss',
    'link_conf_loss', 'pos_link_conf_loss', 'neg_link_conf_loss',
    #'num_pos_seg', 'num_neg_seg', 
    #'num_pos_link', 'num_neg_link',
    'seg_fmeasure', 'link_fmeasure',
    'seg_precision', 'seg_recall',
    'link_precision', 'link_recall',
    # TBPP
    'conf_loss', 'loc_loss',
    'precision', 'recall', 'fmeasure'
]
#names = None

#limits = [100, np.inf]
#limits = [0, 14000]
#limits = [14000, np.inf]
limits = [400, np.inf]
limits = None
#limits = [3000,np.inf]
#limits = [100, 1000]

plot_log(checkdir+'/log.json', names, limits, window_length=2000)

### History

In [ ]:
with open(checkdir+'/history.json','r') as f:
    hist = json.load(f)
#print(list(hist.keys()))
epochs = np.arange(1,len(hist['loss'])+1)
plt.figure(figsize=(8,4))
plt.plot(epochs, hist['loss'])
plt.plot(epochs, hist['val_loss'])
plt.title('loss')
plt.legend(['training','validation'])
#plt.xlim([0.0, 8.0])
ax = plt.gca()
ticks = np.arange(len(hist['loss']))+1
ax.set_xticks(ticks)
ax.set_xticklabels(ticks)
plt.grid()
plot_name = 'dsodsl512_sythtext_b'
plt.savefig('plots/%s_loss.pgf' % (plot_name), bbox_inches='tight')
plt.show()

In [ ]:
# SSD
signals = ['pos_conf_loss', 'neg_conf_loss', 'pos_loc_loss']

# SegLink
signals = ['pos_seg_conf_loss', 'neg_seg_conf_loss', 'seg_loc_loss', 'pos_link_conf_loss', 'neg_link_conf_loss']
#signals = ['num_pos_seg', 'num_neg_seg', 'num_pos_link', 'num_neg_link']
signals = ['seg_conf_loss', 'seg_loc_loss', 'link_conf_loss']
signals = ['seg_fmeasure', 'link_fmeasure', 'seg_precision', 'seg_recall', 'link_precision', 'link_recall']

signals = ['seg_precision', 'seg_recall', 'seg_fmeasure']
signals = ['link_precision', 'link_recall', 'link_fmeasure']

signals = ['precision', 'recall', 'fmeasure']


#signals = ['pos_conf_loss', 'neg_conf_loss', 'pos_loc_loss']

fig, axs = plt.subplots(1, len(signals), figsize=(20,3))
for i, s in enumerate(signals):
    if s not in hist.keys():
        print('missing %s' %(s))
        continue
    axs[i].plot(epochs, hist[s])
    axs[i].plot(epochs, hist['val_'+s])
    axs[i].set_title(s)
    if s.split('_')[-1] in ['precision', 'recall', 'fmeasure']:
        axs[i].set_ylim([0,1])
    ticks = np.arange(len(hist['loss']))+1
    axs[i].set_xticks(ticks)
    axs[i].set_xticklabels(ticks)
    axs[i].grid()
#plt.savefig('plots/%s_links.pgf' % (plot_name), bbox_inches='tight')
plt.show()

### Compare Log

In [ ]:
# SegLink
checkdir1 = './checkpoints/201711132011_dsodsl512_synthtext' # 2 epochs, OHNM
checkdir2 = './checkpoints/201805161634_dsodsl512_synthtext' # 2 epochs, Focal Loss
#checkdir1 = './checkpoints/201806021007_dsodsl512_synthtext' # 24 eopchs, 1/4 of training set

#plot_log(checkdir1+'/log.json', ['loss'], None, window_length=2000, log_file_compare=checkdir2+'/log.json')

In [ ]:
# TextBoxes++
checkdirs = !ls -1d ./checkpoints/*tbpp512fl_*
print(checkdirs.n, '\n')

checkdir1, checkdir2 = checkdirs[0], checkdirs[3]
checkdir1, checkdir2 = checkdirs[3], checkdirs[4]

plot_log(checkdir1+'/log.json', ['loss', 'conf_loss', 'loc_loss', 'precision', 'recall', 'fmeasure'], [0*10000,np.inf], 
         log_file_compare=checkdir2+'/log.json',
         window_length=2500)

### Log playground

In [ ]:
limits = None

with open(checkdir+'/log.json','r') as f:
    data = f.readlines()

keys = json.loads(data[0]).keys()
d = {k:[] for k in keys}

if names is None:
    names = [k for k in keys if k not in ['epoch', 'batch', 'iteration']]

for i, line in enumerate(data):
    if not limits == None and (i < limits[0] or i > limits[1]):
        continue
    dat = json.loads(line)
    for k in keys:
        d[k].append(dat[k])

d = {k:np.array(d[k]) for k in keys}

In [ ]:
fstr = '%-20s %-6s %-6s %-6s %12s %12s'
print(fstr % ('name', 'len', 'nan', 'inf', 'min', 'max'))
for k,v in d.items():
    print(fstr % (
        k, 
        len(v),
        np.any(np.isnan(v)), 
        np.any(np.logical_not(np.isfinite(v))),
        round(np.min(v), 6),
        round(np.max(v), 6),
    ))

In [ ]:
plt.figure()
plt.plot(d['batch']+d['epoch']*max(d['batch'])-d['iteration'])
plt.show()
d['iteration'][-1]

In [ ]:
f = !df -h 
print(f[0], f[3], sep='\n')
f = !nvidia-smi
print('', f[5], f[8], sep='\n')